# Service deployment using TrueFoundry


##### This notebook demonstrates a demo of how you can deploy your first service with TrueFoundry.

---


After you complete the guide, you will have a successfully deployed model. Your deployed API will look like this:

![](https://files.readme.io/e1affc0-Screenshot_2022-11-11_at_5.07.48_PM.png)


## Project structure

To complete this guide, you are going to review the following **files**:

- `app.py` : contains our inference and FastAPI code
- `iris_classifier.joblib` : the model file
- `deploy.py`: contains our deployment code
- `requirements.txt` : contains our dependencies

Your **final file structure** is going to look like this:

```
.
├── app.py
├── iris_classifier.joblib
├── deploy.py
└── requirements.txt
```

As you can see, all the following files are created in the same folder/directory

# Setup

Let's first setup all the things we need to deploy our service.

- Signup or Login on TrueFoundry
- Setup Workspace


Let's start with installing truefoundry.


In [ ]:
%pip install -U "truefoundry>=0.5.9,<0.6.0"

**Login into TrueFoundry**

In order to login run the cell below. Host can be found from the TrueFoundry UI as shown below like https://app.truefoundry.com

![image.png](../common/images/host.png)

In [ ]:
!tfy login --host "<Host name of TrueFoundry UI. e.g. https://company.truefoundry.cloud>"

**Select the `Workspace` in which you want to deploy your application.**

Once you run the cell below you will get a prompt to enter your Workspace FQN. Follow the docs to

**Create a Workspace**: https://docs.truefoundry.com/docs/key-concepts#creating-a-workspace

**Get Existing Workspace FQN**: https://docs.truefoundry.com/docs/key-concepts#getting-workspace-fqn

In [ ]:
import click

In [ ]:
WORKSPACE_FQN = click.prompt(
    "Enter the Workspace FQN",
    type=str,
)
print(f"\nWorkspace FQN set to {WORKSPACE_FQN!r}")

# Step 1: Fetch model

## Model details

For this guide, we have already trained a model. The given model has been trained on _iris dataset_ and uploaded to [GitHub](https://github.com/truefoundry/getting-started-examples/raw/b0ec188da047b75b334ea544560b583ab3b97510/deploy-ml-model/iris_classifier.joblib).

> **Attributes** :
> sepal length in cm, sepal width in cm, petal length in cm, petal width in cm
>
> **Predicted Attribute** :  
> class of iris plant (one of the following - Iris Setosa, Iris Versicolour, Iris Virginica)

## Download instructions

Download the model from the following [link](https://github.com/truefoundry/getting-started-examples/raw/b0ec188da047b75b334ea544560b583ab3b97510/deploy-ml-model/iris_classifier.joblib).  
Then move the model in your dev directory we created.

Afterwards, your directory should look like this :

```
.
└── iris_classifier.joblib
```


# Step 2: Implement the Inference Service code.

The first step is to create a web API and deploy the model.  
For this we are going to use [FastAPI](https://fastapi.tiangolo.com/) for this. FastAPI is a modern, intuitive web framework for building web APIs in python.

Create the `app.py` and `requirements.txt` files in the same directory where the model is stored.

```
.
├── iris_classifier.joblib
├── app.py
└── requirements.txt
```


### **`app.py`**


In [ ]:
%%writefile app.py
import os
import joblib
import pandas as pd
from fastapi import FastAPI

model = joblib.load("iris_classifier.joblib")
app = FastAPI(docs_url="/", root_path=os.getenv("TFY_SERVICE_ROOT_PATH"))

@app.post("/predict")
def predict(
    sepal_length: float, sepal_width: float, petal_length: float, petal_width: float
):
    data = dict(
        sepal_length=sepal_length,
        sepal_width=sepal_width,
        petal_length=petal_length,
        petal_width=petal_width,
    )
    prediction = int(model.predict(pd.DataFrame([data]))[0])
    return {"prediction": prediction}

Click on this [link](https://docs.truefoundry.com/recipes/create-a-fastapi-service-code-to-deploy-model) to understand the **`app.py`**:


In [ ]:
%%writefile requirements.txt
fastapi==0.114.0
uvicorn==0.30.6
scikit-learn==1.5.0
joblib==1.3.2
pandas==2.2.2
numpy==1.26.4

# Step 3: Deploying the Inference App

You can deploy services on TrueFoundry programmatically via our **Python SDK**.

Create a `deploy.py`, after which our file structure will look like this:

**File Structure**

```Text Text
.
├── iris_classifier.joblib
├── app.py
├── deploy.py
└── requirements.txt
```

### **`deploy.py`**


In [ ]:
%%writefile deploy.py
import argparse
import logging
from truefoundry.deploy import Build, PythonBuild, Service, Resources, Port, LocalSource

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s [%(name)s] %(levelname)-8s %(message)s"
)


def str_or_none(value):
    return None if not value or value == "None" else value

parser = argparse.ArgumentParser()
parser.add_argument(
    "--name", 
    required=False, 
    default="iris-classifier-svc",
    type=str, 
    help="Name of the application."
)
parser.add_argument(
    "--workspace_fqn",
    "--workspace-fqn",
    required=True,
    type=str,
    help="FQN of the workspace where application will be deployed.",
)
parser.add_argument(
    "--host",
    required=True,
    type=str,
    help="Host where the application will be available for access. Ex:- my-app.my-org.com",
)
parser.add_argument(
    "--path",
    required=False,
    default=None,
    type=str_or_none,
    help="Path in addition to the host where the application will be available for access. Eg: my-org.com/my-path",
)
args = parser.parse_args()

service = Service(
    name=args.name,
    # Define how to build your code into a Docker image
    image=Build(
        # `LocalSource` helps specify the details of your local source code.
        build_source=LocalSource(local_build=False),
        # `PythonBuild` helps specify the details of your Python Code.
      	# These details will be used to templatize a DockerFile to build your Docker Image
        build_spec=PythonBuild(
            python_version="3.11",
            command="uvicorn app:app --port 8000 --host 0.0.0.0",
            requirements_path="requirements.txt",
        )
    ),
    # Set the ports your server will listen on
    ports=[
        # Providing a host and path value depends on the base domain urls configured in the cluster settings.
        # You can learn how to find the base domain urls available to you https://docs.truefoundry.com/docs/define-ports-and-domains#identifying-available-domains
        Port(port=8000, host=args.host, path=args.path)
    ],
    # Define the resource constraints.
    #
    # Requests are the minimum amount of resources that a container needs to run.
    # Limits are the maximum amount of resources that a container can use.
    resources=Resources(
        cpu_request=0.1,
        cpu_limit=0.1,
        memory_request=500,
        memory_limit=500,
    ),
    # Define environment variables that your Service will have access to
    env={"UVICORN_WEB_CONCURRENCY": "1", "ENVIRONMENT": "dev"},
    labels={"tfy_openapi_path": "openapi.json"},
)
service.deploy(workspace_fqn=args.workspace_fqn, wait=False)

We will need a endpoint to access the deployed service. This host should follow the base domain url configured in the cluster.

Please refer to following docs to get the base domain url to make your endpoint:

https://docs.truefoundry.com/docs/define-ports-and-domains#identifying-available-domains

In [ ]:
SERVICE_HOST = click.prompt(
    "Enter the host for the Service endpoint (e.g. my-service.org-domain.com OR org-domain.com)",
    type=str,
)
SERVICE_PATH = click.prompt(
    "Optionally, enter the path for Service endpoint (e.g. /my-service/). You can leave this blank if your domain supports subdomains: ",
    type=str,
    default='',
)
SERVICE_PATH = f"/{SERVICE_PATH.strip('/')}/" if SERVICE_PATH else None

print(f"\nService Host set to {SERVICE_HOST!r}")
print(f"\nService Path set to {SERVICE_PATH!r}")

Now to deploy our FastAPI Service run the command below


In [ ]:
!python deploy.py --workspace_fqn $WORKSPACE_FQN --host $SERVICE_HOST --path $SERVICE_PATH

Once the build is complete, you will see a link to the dashboard after a message like <br>
`You can find the application on the dashboard:-`.

Click on the link to access the deployment dashboard.
